# Chapter 10 데이터 집계와 그룹 연산

10.1 GroupBy 메카닉

In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.DataFrame({ 'key1': ['a','a','b','b','a'],
                  'key2': ['one','two','one','two','one'],
                  'data1':np.random.randn(5),
                  'data2':np.random.randn(5)})

In [6]:
df

,key1,key2,data1,data2
0,a,one,0.068093,0.692730
1,a,two,-2.257767,1.206317
2,b,one,0.898736,0.480265
3,b,two,0.182520,0.172406
4,a,one,0.525635,-0.198987


In [7]:
grouped = df['data1'].groupby(df['key1'])  # data1의 값을 key값으로 분류한다

In [8]:
grouped

In [9]:
grouped.mean()

key1
a   -0.554680
b    0.540628
Name: data1, dtype: float64

In [13]:
means = df['data1'].groupby([df['key1'],df['key2']]).mean()

In [14]:
means

key1  key2
a     one     0.296864
      two    -2.257767
b     one     0.898736
      two     0.182520
Name: data1, dtype: float64

In [15]:
means.unstack()    # 표시하는 형태만 바뀐건가

key2,one,two
key1,,
a,0.296864,-2.257767
b,0.898736,0.182520


In [16]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])

In [17]:
years = np.array([2005,2005,2006,2005,2006])

In [18]:
df['data1'].groupby([states,years]).mean()

California  2005   -2.257767
            2006    0.898736
Ohio        2005    0.125307
            2006    0.525635
Name: data1, dtype: float64

In [19]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.554680,0.566687
b,0.540628,0.326336


In [20]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.296864  0.246872
     two  -2.257767  1.206317
b    one   0.898736  0.480265
     two   0.182520  0.172406

In [21]:
df.groupby([states,years]).mean()

data1     data2
California 2005 -2.257767  1.206317
           2006  0.898736  0.480265
Ohio       2005  0.125307  0.432568
           2006  0.525635 -0.198987

In [22]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.554680,0.566687
b,0.540628,0.326336


In [23]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.296864  0.246872
     two  -2.257767  1.206317
b    one   0.898736  0.480265
     two   0.182520  0.172406

In [25]:
df.groupby(['key1','key2']).size()    # 그룹의 크기를 반환

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

10.1.1 그룹간 순회하기

In [27]:
for name, group in df.groupby('key1'):  # key1의 그룹이름과 데이터 묶음을 튜플 반환
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.068093  0.692730
1    a  two -2.257767  1.206317
4    a  one  0.525635 -0.198987
b
  key1 key2     data1     data2
2    b  one  0.898736  0.480265
3    b  two  0.182520  0.172406


In [29]:
for (k1, k2), group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.068093  0.692730
4    a  one  0.525635 -0.198987
('a', 'two')
  key1 key2     data1     data2
1    a  two -2.257767  1.206317
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.898736  0.480265
('b', 'two')
  key1 key2    data1     data2
3    b  two  0.18252  0.172406


In [30]:
pieces = dict(list(df.groupby('key1'))) # key1의 네임값을 키로, 데이터를 사전형태로 저장

In [31]:
pieces['b']

,key1,key2,data1,data2
2,b,one,0.898736,0.480265
3,b,two,0.182520,0.172406


In [32]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [35]:
grouped = df.groupby(df.dtypes, axis=1) # 데이터 타입을 키로 묶어라

In [39]:
grouped

In [37]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.068093  0.692730
1 -2.257767  1.206317
2  0.898736  0.480265
3  0.182520  0.172406
4  0.525635 -0.198987
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


10.1.2 컬럼이나 컬럼의 일부만 선택하기

In [40]:
df.groupby(['key1','key2'])[['data2']].mean()   # data2의 값만 뽑아서 평균

data2
key1 key2          
a    one   0.246872
     two   1.206317
b    one   0.480265
     two   0.172406

In [41]:
s_grouped = df.groupby(['key1','key2'])['data2']

In [42]:
s_grouped

In [43]:
s_grouped.mean()   # data2의 값

key1  key2
a     one     0.246872
      two     1.206317
b     one     0.480265
      two     0.172406
Name: data2, dtype: float64

10.1.3 사전과  Series에서 그룹핑하기

In [44]:
people = pd.DataFrame(np.random.randn(5,5),
                     columns = ['a','b','c','d','e'],
                     index =['Joe','Steve','Wes','Jim','Travis'])

In [45]:
people

,a,b,c,d,e
Joe,-0.476517,-0.143855,0.064097,0.299877,-0.259269
Steve,1.864387,0.902184,2.054591,-0.055881,-0.695653
Wes,-1.111707,1.626643,-0.323752,1.712698,-0.183995
Jim,0.486319,-0.860503,-0.407235,-1.455631,-0.490238
Travis,-0.910624,-1.017722,0.092981,-0.034991,0.179152


In [47]:
people.iloc[2:3,[1,2]] = np.nan    # 2포함 3 미포함 == WES, wes의 1,2 

In [48]:
people

,a,b,c,d,e
Joe,-0.476517,-0.143855,0.064097,0.299877,-0.259269
Steve,1.864387,0.902184,2.054591,-0.055881,-0.695653
Wes,-1.111707,NaN,NaN,1.712698,-0.183995
Jim,0.486319,-0.860503,-0.407235,-1.455631,-0.490238
Travis,-0.910624,-1.017722,0.092981,-0.034991,0.179152


In [49]:
mapping = {'a':'red','b':'red','c':'blue',
          'd':'blue','e':'red','f':'orange'}

In [50]:
by_column = people.groupby(mapping,axis=1)   # a,b,e ->red  || c,d, ->blue

In [52]:
by_column.sum()

,blue,red
Joe,0.363974,-0.879642
Steve,1.998710,2.070919
Wes,1.712698,-1.295702
Jim,-1.862866,-0.864422
Travis,0.057990,-1.749194


In [54]:
mapping

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [56]:
map_series = pd.Series(mapping)

In [57]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [60]:
people.groupby(map_series,axis=1).count() # 위에 그거  sum -> count

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


10.1.4 함수로 그룹핑하기

In [61]:
people.groupby(len).sum()  # 이름 글자수를 키값으로

,a,b,c,d,e
3,-1.101906,-1.004358,-0.343138,0.556944,-0.933502
5,1.864387,0.902184,2.054591,-0.055881,-0.695653
6,-0.910624,-1.017722,0.092981,-0.034991,0.179152


In [62]:
people.groupby(len).size()

3    3
5    1
6    1
dtype: int64

In [63]:
key_list = ['one','one','one','two','two']

In [65]:
people.groupby([len, key_list]).min() # Joe,Wes -> 3,one

a         b         c         d         e
3 one -1.111707 -0.143855  0.064097  0.299877 -0.259269
  two  0.486319 -0.860503 -0.407235 -1.455631 -0.490238
5 one  1.864387  0.902184  2.054591 -0.055881 -0.695653
6 two -0.910624 -1.017722  0.092981 -0.034991  0.179152

10.1.5 색인 단계로 그룹핑하기

In [66]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                    [1,3,5,1,3]],
                                   names =['cty','tenor'])

In [68]:
hier_df = pd.DataFrame(np.random.randn(4,5), columns = columns)

In [69]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -2.121211  1.889083  0.420846 -0.505379  0.107154
1     -0.361071  1.517286  0.659553  0.213930  0.144302
2      0.520006  0.013288  1.417119 -0.443228 -0.468925
3     -0.954727  0.663236 -0.533046  1.625742  1.034644

In [70]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [71]:
hier_df.groupby('cty',axis=1).count()    #level을 왜 굳이 사용하는거지?

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


10.2 데이터 집계

In [72]:
df  #나 왜 책이랑 반대지

,key1,key2,data1,data2
0,a,one,0.068093,0.692730
1,a,two,-2.257767,1.206317
2,b,one,0.898736,0.480265
3,b,two,0.182520,0.172406
4,a,one,0.525635,-0.198987


In [75]:
grouped = df.groupby('key1')

In [76]:
grouped['data1'].quantile(0.9)    # 대체 왜 저런값이나오는거야

key1
a    0.434127
b    0.827114
Name: data1, dtype: float64

In [77]:
def peak_to_peak(arr):
    return arr.max() - arr.min() # max-min 함수 생성

In [78]:
grouped.agg(peak_to_peak)  # 함수 적용을 위해 agg 사용한건가

,data1,data2
key1,,
a,2.783402,1.405304
b,0.716216,0.307859


In [81]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.554680  1.492554 -2.257767 -1.094837  0.068093  0.296864   
b      2.0  0.540628  0.506441  0.182520  0.361574  0.540628  0.719682   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.525635   3.0  0.566687  0.711080 -0.198987  0.246872  0.692730   
b     0.898736   2.0  0.326336  0.217689  0.172406  0.249371  0.326336   

                          
           75%       max  
key1                      
a     0.949524  1.206317  
b     0.403300  0.480265

10.2.1 컬럼에 여러 가지 함수 적용하기

In [84]:
tips = pd.read_csv('examples/tips.csv')

In [85]:
tips['tip_pct'] = tips['tip']/tips['total_bill']

In [86]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [87]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [93]:
grouped = tips.groupby(['day','smoker'])

In [94]:
grouped.mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [95]:
grouped_pct = grouped['tip_pct']

In [96]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [97]:
grouped_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [98]:
grouped_pct.agg([('foo','mean'),('bar',np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [99]:
functions = ['count','mean','max']

In [101]:
result = grouped['tip_pct','total_bill'].agg(functions)

D:\Download\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [102]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [103]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [104]:
ftuples =[('Durchschnitt','mean'),('Abweichung',np.var)]

In [105]:
grouped['tip_pct','total_bill'].agg(ftuples)

D:\Download\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [106]:
grouped.agg({'tip':np.max,'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [107]:
grouped.agg({'tip_pct':['min','max','mean','std'],
            'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

10.2.2 색인되지 않은 형태로 집계된 데이터 반환하기

In [108]:
tips.groupby(['day','smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


10.3 Apply:일반적인 분리-적용-병합

In [109]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [110]:
top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [111]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [112]:
tips.groupby(['smoker','day']).apply(top, n=1,column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [113]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [114]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [115]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

10.3.1 그룹 색인 생략하기

In [116]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


10.3.2 변위치 분석과 버킷 분석

In [117]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                     'data2':np.random.randn(1000)})

In [118]:
quartiles = pd.cut(frame.data1, 4)

In [119]:
quartiles[:10]

0     (-0.222, 1.581]
1     (-0.222, 1.581]
2    (-3.836, -2.026]
3     (-0.222, 1.581]
4     (-0.222, 1.581]
5     (-0.222, 1.581]
6      (1.581, 3.384]
7    (-2.026, -0.222]
8      (1.581, 3.384]
9     (-0.222, 1.581]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.836, -2.026] < (-2.026, -0.222] < (-0.222, 1.581] < (1.581, 3.384]]

In [120]:
def get_stats(group):
    return {'min':group.min(), 'max':group.max(),
           'count':group.count(),'mean':group.mean()}

In [121]:
grouped = frame.data2.groupby(quartiles)

In [122]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.836, -2.026]",-1.443565,1.531820,27.0,-0.152750
"(-2.026, -0.222]",-2.765510,2.925101,402.0,-0.001404
"(-0.222, 1.581]",-2.873435,3.117329,525.0,0.060278
"(1.581, 3.384]",-1.540906,2.717966,46.0,0.142851


In [123]:
grouping = pd.qcut(frame.data1,10,labels=False)

In [124]:
grouped=frame.data2.groupby(grouping)

In [125]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.189657,2.446640,100.0,-0.075794
1,-2.299055,2.925101,100.0,-0.006860
2,-1.831449,2.551670,100.0,0.059261
3,-2.765510,2.316167,100.0,-0.035751
4,-2.873435,2.157722,100.0,-0.048889
5,-1.757115,2.840941,100.0,-0.020856
6,-2.470563,3.117329,100.0,0.130856
7,-1.686593,3.026177,100.0,0.130185
8,-1.745466,2.244065,100.0,0.127863


10.3.3 예제: 그룹에 따른 값으로 결측치 채우기

In [127]:
s = pd.Series(np.random.randn(6))

In [128]:
s[::2] = np.nan

In [129]:
s

0         NaN
1    1.114169
2         NaN
3   -0.102812
4         NaN
5   -0.653185
dtype: float64

In [130]:
s.fillna(s.mean())

0    0.119391
1    1.114169
2    0.119391
3   -0.102812
4    0.119391
5   -0.653185
dtype: float64

In [131]:
states = ['Ohio','New York','Vermont','Florida','Oregon','Nevada','California','Idaho']

In [132]:
group_key =['East']*4 + ['West']*4

In [133]:
data = pd.Series(np.random.randn(8), index=states)

In [134]:
data

Ohio          0.938135
New York     -2.094039
Vermont       0.272450
Florida      -1.151882
Oregon        1.537314
Nevada        0.341630
California    1.116575
Idaho        -0.638175
dtype: float64

In [135]:
data[['Vermont','Nevada','Idaho']] = np.nan

In [136]:
data

Ohio          0.938135
New York     -2.094039
Vermont            NaN
Florida      -1.151882
Oregon        1.537314
Nevada             NaN
California    1.116575
Idaho              NaN
dtype: float64

In [137]:
data.groupby(group_key).mean()

East   -0.769262
West    1.326945
dtype: float64

In [138]:
fill_mean = lambda g: g.fillna(g.mean())

In [139]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.938135
New York     -2.094039
Vermont      -0.769262
Florida      -1.151882
Oregon        1.537314
Nevada        1.326945
California    1.116575
Idaho         1.326945
dtype: float64

In [141]:
fillvalues={'East':0.5,'West':-1}

In [142]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.938135
New York     -2.094039
Vermont      -0.769262
Florida      -1.151882
Oregon        1.537314
Nevada        1.326945
California    1.116575
Idaho         1.326945
dtype: float64

In [143]:
fill_values = {'East':0.5,'West':-1}

In [144]:
fill_func = lambda g:g.fillna(fill_values[g.name])

In [145]:
data.groupby(group_key).apply(fill_func)

Ohio          0.938135
New York     -2.094039
Vermont       0.500000
Florida      -1.151882
Oregon        1.537314
Nevada       -1.000000
California    1.116575
Idaho        -1.000000
dtype: float64

10.3.4 예제 : 랜덤 표본과 수열

In [149]:
suits =['H','S','C','D']
card_val = (list(range(1,11))+[10]*3)*4
base_names = ['A'] + list(range(2,11)) + ['J','K','Q']
cards = []
for suit in ['H','S','C','D']:
    cards.extend(str(num)+suit for num in base_names)

deck = pd.Series(card_val,index=cards)

In [150]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [151]:
def draw(deck,n=5):
    return deck.sample(n)

In [152]:
draw(deck)

10C    10
2C      2
2D      2
7C      7
9C      9
dtype: int64

In [153]:
get_suit = lambda card: card[-1] # 마지막 글자가 세트

In [154]:
deck.groupby(get_suit).apply(draw,n=2)

C  KC     10
   AC      1
D  10D    10
   KD     10
H  QH     10
   4H      4
S  7S      7
   9S      9
dtype: int64

In [155]:
deck.groupby(get_suit,group_keys=False).apply(draw,n=2)

3C     3
6C     6
6D     6
7D     7
AH     1
9H     9
KS    10
JS    10
dtype: int64

10.3.5 예제: 그룹 가중 평균과 상관관계

In [157]:
df = pd.DataFrame({ 'category':['a','a','a','a',
                               'b','b','b','b'],
                  'data':np.random.randn(8),
                  'weights':np.random.rand(8)})

In [158]:
df

,category,data,weights
0,a,1.150847,0.352165
1,a,-1.298483,0.250750
2,a,-1.365045,0.567432
3,a,0.844536,0.628988
4,b,-0.147992,0.274140
5,b,-0.064309,0.117051
6,b,0.763834,0.433914
7,b,1.096030,0.615177


In [159]:
grouped = df.groupby('category')

In [160]:
get_wavg = lambda g: np.average(g['data'], weights =g['weights'])

In [161]:
grouped.apply(get_wavg)

category
a   -0.090963
b    0.664865
dtype: float64

In [162]:
close_px =pd. read_csv('examples/stock_px_2.csv', parse_dates =True,
                      index_col=0)

In [163]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [164]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [165]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [166]:
rets = close_px.pct_change().dropna()

In [167]:
get_year = lambda x: x.year

In [168]:
by_year = rets.groupby(get_year)

In [169]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [172]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

10.3.6 예제: 그룹상의 선형회귀

In [178]:
import statsmodels.api as sm
def regress(data,yvar,xvars):
    Y = data[yvar]
    X=data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [179]:
by_year.apply(regress,'AAPL',['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


In [ ]:
10.4 피벗테이블과 교차일람표

In [180]:
tips.pivot_table(index=['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [181]:
tips.pivot_table(['tip_pct','size'], index =['time','day'],
                columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [182]:
tips.pivot_table(['tip_pct','size'], index =['time','day'],
                columns ='smoker',margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [183]:
tips.pivot_table('tip_pct',index=['time','smoker'],columns='day',
                aggfunc=len,margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [184]:
tips.pivot_table('tip_pct',index=['time','size','smoker'],columns='day',
                aggfunc='mean',fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

10.4.1 교차일람표

In [185]:
data  # 책이랑 다르네...

Ohio          0.938135
New York     -2.094039
Vermont            NaN
Florida      -1.151882
Oregon        1.537314
Nevada             NaN
California    1.116575
Idaho              NaN
dtype: float64

In [186]:
pd.crosstab(data.Nationality,data.Handedness,margins=True)

AttributeError: 'Series' object has no attribute 'Nationality'

In [188]:
pd.crosstab([tips.time,tips.day], tips.smoker,margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244